In [1]:
Training_dataset = './BraTS_2020/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
Val_dataset = './BraTS_2020/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/'

In [ ]:
import torch
torch.cuda.is_available()

In [3]:
import nibabel as nib
from matplotlib import pyplot as plt

In [ ]:
import nibabel as nib
from matplotlib import pyplot as plt
import os

test_image_flair = nib.load(Training_dataset + 'BraTS20_Training_001/BraTS20_Training_001_flair.nii').get_fdata()
test_image_t1 = nib.load(Training_dataset + 'BraTS20_Training_001/BraTS20_Training_001_t1.nii').get_fdata()
test_image_t1ce = nib.load(Training_dataset + 'BraTS20_Training_001/BraTS20_Training_001_t1ce.nii').get_fdata()
test_image_t2 = nib.load(Training_dataset + 'BraTS20_Training_001/BraTS20_Training_001_t2.nii').get_fdata()
test_mask = nib.load(Training_dataset + 'BraTS20_Training_001/BraTS20_Training_001_seg.nii').get_fdata()

fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1,5, figsize = (20, 10))
slice_w = 25
# slice_number = test_image_flair.shape[0]//2-slice_w
slice_number = 55
print(slice_number)
ax1.imshow(test_image_flair[:,:,slice_number], cmap='gray')
ax1.set_title('Image flair')
ax2.imshow(test_image_t1[:,:,slice_number], cmap='gray')
ax2.set_title('Image t1')
ax3.imshow(test_image_t1ce[:,:,slice_number], cmap='gray')
ax3.set_title('Image t1ce')
ax4.imshow(test_image_t2[:,:,slice_number], cmap='gray')
ax4.set_title('Image t2')
ax5.imshow(test_mask[:,:,slice_number])
ax5.set_title('Mask')

In [ ]:
from skimage.util import montage
from skimage.transform import rotate
import numpy as np

# We can skip 50:-50 slices since there is not much to see
fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
ax1.imshow(montage(np.transpose(test_image_t1[..., 50:-50], (2, 0, 1))), cmap ='gray')

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
ax1.imshow(montage(np.transpose(test_mask[..., 50:-50], (2, 0, 1))), cmap ='gray')

In [7]:
import torch.nn.functional as F
import torch
import torch.nn as nn

class DiceLoss(nn.Module):
    def __init__(self, weight = None, size_average = True):
        super(DiceLoss, self).__init__()
    
    def forward(self, inputs, targets, smooth = 1):
        inputs = F.sigmoid(inputs)
        
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        
        return 1 - dice

In [ ]:
from sklearn.model_selection import train_test_split
import os

train_dir = [i.path for i in os.scandir(Training_dataset) if i.is_dir()]
val_dir = [j.path for j in os.scandir(Val_dataset) if j.is_dir()]

def path_list_into_ids(dirList):
    x = []
    for i in range(0, len(dirList)):
        x.append(dirList[i][dirList[i].rfind('/') + 1:])
    
    return x

train_test_ids = path_list_into_ids(train_dir)
val_ids = path_list_into_ids(val_dir)

# train_test_ids, val_ids = train_test_split(train_and_test_ids, test_size = 0.15)
train_ids, test_ids = train_test_split(train_test_ids, test_size = 0.15)

print(f"Train : {len(train_ids)} | Validation : {len(val_ids)} | Test : {len(test_ids)}")

In [ ]:
def load_dataset(ids, path):
    images = np.zeros((len(ids) * 10, 240, 240), np.float32)
    masks = np.zeros((len(ids) * 10, 240, 240), np.float32)
    
    i = 0
    
    for id in ids:
        t2 = nib.load(f"{path}{id}/{id}_t2.nii").get_fdata()
        seg = nib.load(f"{path}{id}/{id}_seg.nii").get_fdata()
        
        for s in range(50, seg.shape[2]-50, 10):
            images[i] = t2[:, :, s] / t2.max()
            masks[i] = seg[:, :, s] > 0
            i += 1
            
    images = np.expand_dims(images[:i], axis=1)
    masks = np.expand_dims(masks[:i], axis=1)
    
    return images, masks


train_images, train_masks = load_dataset(train_ids, Training_dataset)
val_images, val_masks = load_dataset(val_ids, Val_dataset)

In [ ]:
train_images.shape, val_masks.shape

In [ ]:
plt.subplot(121)
plt.imshow(train_images[100, 0], cmap='gray')
plt.subplot(122)
plt.imshow(train_masks[100, 0], cmap='gray')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

train_dataset = TensorDataset(torch.from_numpy(train_images).type(torch.float32), torch.from_numpy(train_masks).type(torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(torch.from_numpy(val_images).type(torch.float32), torch.from_numpy(val_masks).type(torch.float32))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

In [ ]:
def save_categorized_slices(image_dir, mask_dir, nii_dir, slice_index=78):
    modalities = ['t1', 't1ce', 't2', 'flair']
    patients = [f for f in os.listdir(nii_dir) if not f.startswith('.')]

    for patient in patients:
        patient_folder = os.path.join(nii_dir, patient)
        for modality in modalities:
            image_path = os.path.join(patient_folder, f"{patient}_{modality}.nii")
            nii_image = nib.load(image_path)
            image_data = nii_image.get_fdata()

            # Normalize and save the image slice
            slice_img = image_data[:, :, slice_index]
            img_norm = (slice_img - np.min(slice_img)) / (np.max(slice_img) - np.min(slice_img))
            img_pil = Image.fromarray(np.uint8(img_norm * 255))
            img_pil.save(os.path.join(image_dir, f"{patient}_{modality}.png"))

        # Process and save the mask with separate channels
        mask_path = os.path.join(patient_folder, f"{patient}_seg.nii")
        nii_mask = nib.load(mask_path)
        mask_data = nii_mask.get_fdata()
        slice_mask = mask_data[:, :, slice_index]

        # Create a blank channel for each label
        mask_channels = {1: np.zeros_like(slice_mask), 2: np.zeros_like(slice_mask), 4: np.zeros_like(slice_mask)}
        for label in mask_channels:
            mask_channels[label][slice_mask == label] = 1

        # Combine channels into a single image (stacking)
        combined_mask = np.stack((mask_channels[1], mask_channels[2], mask_channels[4]), axis=-1)
        mask_image = Image.fromarray(np.uint8(combined_mask * 255))
        mask_image.save(os.path.join(mask_dir, f"{patient}_seg.png"))

save_categorized_slices('./Data/image', './Data/mask', './BraTS_2020/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData')